In [9]:

import import_ipynb
# import rice_seed_feature_extraction
import cv2
import os
import shutil
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
import cv2
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
import os
import pandas as pd
from skimage.measure import moments, moments_central, shannon_entropy
from sklearn.cluster import KMeans
import mahotas
from tqdm import tqdm
from scipy.stats import skew, kurtosis
import pywt
from sklearn import svm

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier
import tqdm
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
import os

In [4]:
import albumentations as A

In [ ]:
# def process_directory(base_path, output_dir, split_ratio=0.8):
#     # Create base output directory
#     os.makedirs(output_dir, exist_ok=True)

#     # Iterate through each variety directory
#     for variety in os.listdir(base_path):
#         variety_path = os.path.join(base_path, variety)
#         if os.path.isdir(variety_path):
#             # Create variety directory in output
#             variety_output = os.path.join(output_dir, variety)
#             train_dir = os.path.join(variety_output, 'train')
#             test_dir = os.path.join(variety_output, 'test')
            
#             # Create Positive and Negative directories in train and test
#             train_positive_dir = os.path.join(train_dir, 'Positive')
#             train_negative_dir = os.path.join(train_dir, 'Negative')
#             test_positive_dir = os.path.join(test_dir, 'Positive')
#             test_negative_dir = os.path.join(test_dir, 'Negative')
            
#             os.makedirs(train_positive_dir, exist_ok=True)
#             os.makedirs(train_negative_dir, exist_ok=True)
#             os.makedirs(test_positive_dir, exist_ok=True)
#             os.makedirs(test_negative_dir, exist_ok=True)
            
#             # Iterate through the subdirectories
#             for subdir in os.listdir(variety_path):
#                 subdir_path = os.path.join(variety_path, subdir)
#                 if os.path.isdir(subdir_path):
#                     # Determine if this is a positive or negative directory
#                     is_negative = subdir.lower() == "negative"
                    
#                     # Choose the appropriate destination directories
#                     train_dest_dir = train_negative_dir if is_negative else train_positive_dir
#                     test_dest_dir = test_negative_dir if is_negative else test_positive_dir
#                     label = "Negative" if is_negative else "Positive"
                    
#                     # Collect all image paths for the current subdirectory
#                     images = []
#                     for file in os.listdir(subdir_path):
#                         if file.endswith(('.png', '.jpg', '.jpeg')):
#                             images.append(os.path.join(subdir_path, file))
                    
#                     # Shuffle the image paths to ensure random distribution
#                     random.shuffle(images)

#                     # Split the image paths into training and testing sets
#                     split_index = int(len(images) * split_ratio)
#                     train_images = images[:split_index]
#                     test_images = images[split_index:]

#                     # Copy training images
#                     print(f"Copying {len(train_images)} training images for {variety}/{label} from {subdir}")
#                     for image_path in train_images:
#                         destination_path = os.path.join(train_dest_dir, os.path.basename(image_path))
#                         shutil.copy(image_path, destination_path)

#                     # Copy testing images
#                     print(f"Copying {len(test_images)} testing images for {variety}/{label} from {subdir}")
#                     for image_path in test_images:
#                         destination_path = os.path.join(test_dest_dir, os.path.basename(image_path))
#                         shutil.copy(image_path, destination_path)

#     print("Image copying completed.")



# base_dir = '/home/duyle/Documents/AIL/Rice_photos-master'
# output_dir = '/home/duyle/Documents/AIL/Aug'
# process_directory(base_dir, output_dir)

Copying 788 training images for Thien_uu/Negative from Negative
Copying 197 testing images for Thien_uu/Negative from Negative
Copying 816 training images for Thien_uu/Positive from Thien_uu-8
Copying 205 testing images for Thien_uu/Positive from Thien_uu-8
Copying 1534 training images for Xi23/Negative from Negative
Copying 384 testing images for Xi23/Negative from Negative
Copying 1781 training images for Xi23/Positive from Xi-23
Copying 446 testing images for Xi23/Positive from Xi-23
Copying 1643 training images for Huongthom/Negative from Negative
Copying 411 testing images for Huongthom/Negative from Negative
Copying 1676 training images for Huongthom/Positive from Huong_thom-1
Copying 420 testing images for Huongthom/Positive from Huong_thom-1
Copying 1179 training images for Nep87/Negative from Negative
Copying 295 testing images for Nep87/Negative from Negative
Copying 1119 training images for Nep87/Positive from Nep-87
Copying 280 testing images for Nep87/Positive from Nep-87


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import albumentations as A
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def basic_feature(image):
    # Your existing feature extraction code...
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    blur = cv2.GaussianBlur(image, (3, 3), 0)
    _, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    area = np.count_nonzero(new_img)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    contour = max(contours, key=cv2.contourArea)
    peri = cv2.arcLength(contour, True)
    x, y, w, h = cv2.boundingRect(contour)
    
    length = x + w
    width = y + h
    ratio = length / width if width != 0 else 0
    
    ellipse = cv2.fitEllipse(contour)
    major_axis = max(ellipse[1])
    minor_axis = min(ellipse[1])
    
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    
    sf1 = major_axis / area if area != 0 else 0
    sf2 = minor_axis / area if area != 0 else 0
    sf3 = area / ((0.5 * major_axis) ** 2 * np.pi) if major_axis != 0 else 0
    sf4 = area / (0.5 ** 2 * major_axis * minor_axis * np.pi) if major_axis != 0 and minor_axis != 0 else 0
    
    ed = np.sqrt(4 * area / np.pi) if area != 0 else 0
    ar = major_axis / minor_axis if minor_axis != 0 else 0
    roundness = (4 * area * np.pi) / peri ** 2 if peri != 0 else 0
    Co = ed / major_axis if major_axis != 0 else 0
    solid = area / hull_area if hull_area != 0 else 0

    return {
        "area": area,
        "length": length,
        "width": width,
        "ratio": ratio,
        "major_axis_length": major_axis,
        "minor_axis_length": minor_axis,
        "convex_hull_area": hull_area,
        "convex_hull_perimeter": hull_perimeter,
        "shape_factor_1": sf1,
        "shape_factor_2": sf2,
        "shape_factor_3": sf3,
        "shape_factor_4": sf4,
        "equivalent_diameter": ed,
        "aspect_ratio": ar,
        "perimeter": peri,
        "roundness": roundness,
        "compactness": Co,
        "solidity": solid
    }



Loading images from: /home/duyle/Documents/AIL/Aug/Thien_uu/train/Positive


Loading images: 100%|██████████| 816/816 [00:02<00:00, 336.51it/s]


Loading images from: /home/duyle/Documents/AIL/Aug/Thien_uu/train/Negative


Loading images: 100%|██████████| 788/788 [00:02<00:00, 315.88it/s]


In [25]:
def compute_zernike_moments(image, degree=8): # 25 features
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    _, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    mask = np.zeros(new_img.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], contourIdx=-1, color=(255,), thickness=-1)
    binary = (mask > 0).astype(np.uint8)
    
    (x, y), radius = cv2.minEnclosingCircle(contour)
    x, y = int(x), int(y)
    radius = int(np.ceil(radius))
    x1 = max(x - radius, 0)
    y1 = max(y - radius, 0)
    x2 = x + radius
    y2 = y + radius
    
    cropped_mask = binary[y1:y2, x1:x2]
    
    h, w = cropped_mask.shape
    if h != w:
        size = max(h, w)
        square_mask = np.zeros((size, size), dtype=np.uint8)
        y_offset = (size - h) // 2
        x_offset = (size - w) // 2
        square_mask[y_offset:y_offset+h, x_offset:x_offset+w] = cropped_mask
    else:
        square_mask = cropped_mask
        
    effective_radius = square_mask.shape[0] // 2
    zernike_moments = mahotas.features.zernike_moments(square_mask, effective_radius, degree)
    
    return {f"Zernike_{i}": zernike_moments[i] for i in range(len(zernike_moments))}


# In[2]:


def compute_central_moments(image): # 16 features
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.astype(np.float64)
    M = moments(image)
    mm_central = moments_central(image, center=(M[1, 0] / M[0, 0], M[0, 1] / M[0, 0])).flatten()
    return {f"Central_{i}": mm_central[i] for i in range(len(mm_central))}


# In[3]:


def compute_lbp_feature(image): # 10 features
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(image, R=1, P=8, method="uniform")
    hist, bins = np.histogram(lbp.flatten(), bins=10, range=(0, 10))
    return {f"LBP_{i}": hist[i] for i in range(len(hist))}


# In[4]:


def compute_texture_feature(image): # 4 features
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, bins = np.histogram(image.flatten(), bins=256, range=(0, 256), density=True)
    bins = bins[:-1]
    mean = np.sum(hist * bins)
    std = np.sqrt(np.sum((bins - mean)**2 * hist))
    uniformity = np.sum(hist**2)
    third_moment = np.sum((bins - mean)**3 * hist)
    return {
        "texture_mean": mean,
        "texture_std": std,
        "texture_uniformity": uniformity,
        "texture_third_moment": third_moment
    }


# In[5]:


def compute_color_feature(image): # 33 features
    def entropy(channel):
        return shannon_entropy(channel)
    def waveLet(channel):
        max_level = pywt.dwt_max_level(min(channel.shape), "db4")
        coeffs = pywt.wavedec2(channel, "db4", level=max_level)
        return np.mean(coeffs[0].ravel())
    
    # BGR
    B, G, R = cv2.split(image)
    mean_R, mean_G, mean_B = np.mean(R), np.mean(G), np.mean(B)
    # sqrt_R, sqrt_G, sqrt_B = np.sqrt(mean_R), np.sqrt(mean_G), np.sqrt(mean_B)
    std_R, std_G, std_B = np.std(R), np.std(G), np.std(B)
    skew_R, skew_G, skew_B = skew(R.flatten()), skew(G.flatten()), skew(B.flatten())
    kur_R, kur_G, kur_B = kurtosis(R.flatten()), kurtosis(G.flatten()), kurtosis(B.flatten())
    ent_R, ent_G, ent_B = entropy(R), entropy(G), entropy(B)
    wav_R, wav_G, wav_B = waveLet(R), waveLet(G), waveLet(B)
    
    # HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    mean_h, mean_s, mean_v = np.mean(h), np.mean(s), np.mean(v)
    std_h, std_s, std_v = np.std(h), np.std(s), np.std(v)
    # sqrt_h, sqrt_s, sqrt_v = np.sqrt(mean_h), np.sqrt(mean_s), np.sqrt(mean_v)
    skew_h, skew_s, skew_v = skew(h.flatten()), skew(s.flatten()), skew(v.flatten())
    kur_h, kur_s, kur_v = kurtosis(h.flatten()), kurtosis(s.flatten()), kurtosis(v.flatten())
    ent_h, ent_s, ent_v = entropy(h), entropy(s), entropy(v)
    wav_h, wav_s, wav_v = waveLet(h), waveLet(s), waveLet(v)
    
    # Lab
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    mean_l, mean_a, mean_b = np.mean(l), np.mean(a), np.mean(b)
    std_l, std_a, std_b = np.std(l), np.std(a), np.std(b)
    # sqrt_l, sqrt_a, sqrt_b = np.sqrt(mean_l), np.sqrt(mean_a), np.sqrt(mean_b)
    skew_l, skew_a, skew_b = skew(l.flatten()), skew(a.flatten()), skew(b.flatten())
    kur_l, kur_a, kur_b = kurtosis(l.flatten()), kurtosis(a.flatten()), kurtosis(b.flatten())
    ent_l, ent_a, ent_b = entropy(l), entropy(a), entropy(b)
    wav_l, wav_a, wav_b = waveLet(l), waveLet(a), waveLet(b)
    
    # YCbCr
    ycrcb_img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb_img)
    mean_y, mean_cr, mean_cb = np.mean(y), np.mean(cr), np.mean(cb)
    std_y, std_cr, std_cb = np.std(y), np.std(cr), np.std(cb)
    # sqrt_y, sqrt_cr, sqrt_cb = np.sqrt(mean_y), np.sqrt(mean_cr), np.sqrt(mean_cb)
    skew_y, skew_cr, skew_cb = skew(y.flatten()), skew(cr.flatten()), skew(cb.flatten())
    kur_y, kur_cr, kur_cb = kurtosis(y.flatten()), kurtosis(cr.flatten()), kurtosis(cb.flatten())
    ent_y, ent_cr, ent_cb = entropy(y), entropy(cr), entropy(cb)
    wav_y, wav_cr, wav_cb = waveLet(y), waveLet(cr), waveLet(cb)
    
    # XYZ
    xyz_img = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    X, Y, Z = cv2.split(xyz_img)
    mean_X, mean_Y, mean_Z = np.mean(X), np.mean(Y), np.mean(Z)
    std_X, std_Y, std_Z = np.std(X), np.std(Y), np.std(Z)
    skew_X, skew_Y, skew_Z = skew(X.flatten()), skew(Y.flatten()), skew(Z.flatten())
    kur_X, kur_Y, kur_Z = kurtosis(X.flatten()), kurtosis(Y.flatten()), kurtosis(Z.flatten())
    ent_X, ent_Y, ent_Z = entropy(X), entropy(Y), entropy(Z)
    wav_X, wav_Y, wav_Z = waveLet(X), waveLet(Y), waveLet(Z)
    
    return {"mean_r": mean_R, "mean_g": mean_G, "mean_B": mean_B,
            # "sqrt_r": sqrt_R, "sqrt_g": sqrt_G, "sqrt_B": sqrt_B,
            "std_r": std_R, "std_g": std_G, "std_B": std_B,
            "skew_r": skew_R, "skew_g": skew_G, "skew_B": skew_B,
            "kur_r": kur_R, "kur_g": kur_G, "kur_B": kur_B,
            "ent_r": ent_R, "ent_g": ent_G, "ent_B": ent_B,
            "wav_r": wav_R, "wav_g": wav_G, "wav_B": wav_B,
            
            "mean_h": mean_h, "mean_s": mean_s, "mean_v": mean_v,
            "std_h": std_h, "std_s": std_s, "std_v": std_v,
            # "sqrt_h": sqrt_h, "sqrt_s": sqrt_s, "sqrt_v": sqrt_v,
            "skew_h": skew_h, "skew_s": skew_s, "skew_v": skew_v,
            "kur_h": kur_h, "kur_s": kur_s, "kur_v": kur_v,
            "ent_h": ent_h, "ent_s": ent_s, "ent_v": ent_v,
            "wav_h": wav_h, "wav_s": wav_s, "wav_v": wav_v,
            
            "mean_l": mean_l, "mean_a": mean_a, "mean_b": mean_b,
            "std_l": std_l, "std_a": std_a, "std_b": std_b,
            # "sqrt_l": sqrt_l, "sqrt_a": sqrt_a, "sqrt_b": sqrt_b,
            "skew_l": skew_l, "skew_a": skew_a, "skew_b": skew_b,
            "kur_l": kur_l, "kur_a": kur_a, "kur_b": kur_b,
            "ent_l": ent_l, "ent_a": ent_a, "ent_b": ent_b,
            "wav_l": wav_l, "wav_a": wav_a, "wav_b": wav_b,
            
            "mean_y": mean_y, "mean_cb": mean_cb, "mean_cr": mean_cr,
            "std_y": std_y, "std_cb": std_cb, "std_cr": std_cr,
            # "sqrt_y": sqrt_y, "sqrt_cb": sqrt_cb, "sqrt_cr": sqrt_cr,
            "skew_y": skew_y, "skew_cb": skew_cb, "skew_cr": skew_cr,
            "kur_y": kur_y, "kur_cb": kur_cb, "kur_cr": kur_cr,
            "ent_y": ent_y, "ent_cb": ent_cb, "ent_cr": ent_cr,
            "wav_y": wav_y, "wav_cb": wav_cb, "wav_cr": wav_cr,
            
            "mean_x": mean_X, "mean_Y": mean_Y, "mean_z": mean_Z,
            "std_x": std_X, "std_Y": std_Y, "std_z": std_Z,
            "skew_x": skew_X, "skew_Y": skew_Y, "skew_z": skew_Z,
            "kur_x": kur_X, "kur_Y": kur_Y, "kur_z": kur_Z,
            "ent_x": ent_X, "ent_Y": ent_Y, "ent_z": ent_Z,
            "wav_x": wav_X, "wav_Y": wav_Y, "wav_z": wav_Z
            }


# In[6]:


def basic_feature(image): # 8 features
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    area = np.count_nonzero(new_img)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    peri = cv2.arcLength(contour, True)
    x, y, w, h = cv2.boundingRect(contour)
    
    length = x + w
    width = y + h
    ratio = length / width
    
    ellipse = cv2.fitEllipse(contour)
    
    major_axis = max(ellipse[1])
    minor_axis = min(ellipse[1])
    
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    
    sf1 = major_axis / area
    sf2 = minor_axis / area
    sf3 = area / ((0.5 * major_axis)**2 * np.pi)
    sf4 = area / (0.5**2 * major_axis * minor_axis * np.pi)
    
    ed = np.sqrt(4 * area / np.pi)
    ar = major_axis / minor_axis
    roundness = (4 * area * np.pi) / peri**2
    Co = ed / major_axis
    solid = area / hull_area
    
    return {
        "area": area,
        "length": length,
        "width": width,
        "ratio": ratio,
        "major_axis_length": major_axis,
        "minor_axis_length": minor_axis,
        "convex_hull_area": hull_area,
        "convex_hull_perimeter": hull_perimeter,
        "shape_factor_1": sf1,
        "shape_factor_2": sf2,
        "shape_factor_3": sf3,
        "shape_factor_4": sf4,
        "equivalent_diameter": ed,
        "aspect_ratio": ar,
        "perimeter": peri,
        "roundness": roundness,
        "compactness": Co,
        "solidity": solid
    }


# In[7]:


def compute_glcm_descriptor(image): # 16 features
    if image is None:
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Configuration 
    distance = [3]  
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0°, 45°, 90°, 135°
    properties = ['contrast', 'correlation', 'energy', 'homogeneity']  
    
    glcm = graycomatrix(image, distances=distance, angles=angles, symmetric=True, normed=True)
    
    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)
    
    return np.array(features)


# In[8]:


def compute_percentile_features(image):
    features = {}
    
    percentiles = [5, 25, 50, 75, 95]
    
    for color_space, prefix, channel_names in [
        (None, 'bgr', ['B', 'G', 'R']),  
        (cv2.COLOR_BGR2HSV, 'hsv', ['H', 'S', 'V']),
        (cv2.COLOR_BGR2LAB, 'lab', ['L', 'A', 'B']),
        (cv2.COLOR_BGR2YCrCb, 'ycrcb', ['Y', 'Cr', 'Cb']),
        (cv2.COLOR_BGR2XYZ, 'xyz', ['X', 'Y', 'Z'])
    ]:
        if color_space is None:
            converted = image  
        else:
            converted = cv2.cvtColor(image, color_space)
        
        channels = cv2.split(converted)
        
        for i, channel_name in enumerate(channel_names):
            for p in percentiles:
                value = np.percentile(channels[i], p)
                features[f'pf_{prefix}_{channel_name.lower()}_p{p}'] = value
    
    return features

def compute_color_variance_ratios(image):
    features = {}
    
    # BGR
    B, G, R = cv2.split(image)
    var_B, var_G, var_R = np.var(B), np.var(G), np.var(R)
    
    # Variance ratios
    features['var_ratio_R_G'] = var_R / (var_G + 1e-7)
    features['var_ratio_R_B'] = var_R / (var_B + 1e-7)
    features['var_ratio_G_B'] = var_G / (var_B + 1e-7)
    
    # HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv_img)
    var_H, var_S, var_V = np.var(H), np.var(S), np.var(V)
    
    features['var_ratio_H_S'] = var_H / (var_S + 1e-7)
    features['var_ratio_H_V'] = var_H / (var_V + 1e-7)
    features['var_ratio_S_V'] = var_S / (var_V + 1e-7)
    
    # Lab
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    L, A, B_lab = cv2.split(lab_img)
    var_L, var_A, var_B_lab = np.var(L), np.var(A), np.var(B_lab)
    
    features['var_ratio_L_A'] = var_L / (var_A + 1e-7)
    features['var_ratio_L_B'] = var_L / (var_B_lab + 1e-7)
    features['var_ratio_A_B'] = var_A / (var_B_lab + 1e-7)
    
    return features

def compute_color_range_features(image):
    features = {}
    
    for color_space, prefix, channel_names in [
        (None, 'bgr', ['B', 'G', 'R']),  
        (cv2.COLOR_BGR2HSV, 'hsv', ['H', 'S', 'V']),
        (cv2.COLOR_BGR2LAB, 'lab', ['L', 'A', 'B']),
        (cv2.COLOR_BGR2YCrCb, 'ycrcb', ['Y', 'Cr', 'Cb']),
        (cv2.COLOR_BGR2XYZ, 'xyz', ['X', 'Y', 'Z'])
    ]:
        if color_space is None:
            converted = image  
        else:
            converted = cv2.cvtColor(image, color_space)
        
        channels = cv2.split(converted)
        
        for i, channel_name in enumerate(channel_names):
            channel = channels[i]
            
            # Range 
            features[f'range_{prefix}_{channel_name.lower()}_range'] = np.max(channel) - np.min(channel)
            
            # Interquartile range 
            features[f'iqr_{prefix}_{channel_name.lower()}_iqr'] = np.percentile(channel, 75) - np.percentile(channel, 25)

            
            # Mode + mode concentration
            hist, bin_edges = np.histogram(channel, bins=256, range=(0, 256))
            mode_bin = np.argmax(hist)
            mode_value = (bin_edges[mode_bin] + bin_edges[mode_bin + 1]) / 2
            mode_concentration = hist[mode_bin] / np.sum(hist)
            
            features[f'mv_{prefix}_{channel_name.lower()}_mode'] = mode_value
            features[f'mc_{prefix}_{channel_name.lower()}_mode_conc'] = mode_concentration
    
    return features

In [39]:
def extract_all_features(image):
    features = {}

    lbp_hist = compute_lbp_feature(image)
    features.update(lbp_hist) 
    
    texture_features = compute_texture_feature(image)
    features.update(texture_features) 

    color_features = compute_color_feature(image)
    features.update(color_features) 

    zernike_features = compute_zernike_moments(image, degree=8)
    features.update(zernike_features)
    
    shape_features = basic_feature(image)
    features.update(shape_features)

    glcm_features = compute_glcm_descriptor(image)
    if glcm_features is not None:
        for i, val in enumerate(glcm_features):
            features[f'GLCM_{i}'] = val
    
    var_color = compute_color_variance_ratios(image)
    features.update(var_color)
    
    percentile_features = compute_percentile_features(image)
    features.update(percentile_features)
    
    color_range_features = compute_color_range_features(image)
    features.update(color_range_features)

    return features
def augment_image(image):
    # Define your augmentations using Albumentations
    transform = A.Compose([
        A.Rotate(limit=30, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussianBlur(blur_limit=(3, 5))
    ])
    return transform(image=image)['image']
def process_directory(base_path):
    all_data = []
    image_paths = []
    
    # Walk through the directory and gather image paths and labels
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                label = 0 if 'negative' in root.lower() else 1
                image_paths.append((os.path.join(root, file), label))
    
    # Process each image and extract features
    for image_path, label in tqdm(image_paths, desc="Processing Images"):
        image = cv2.imread(image_path)
        if image is None:
            continue  
            
        features = extract_all_features(image)
        features["Label"] = label  # Add the label to features

        all_data.append(features)
        for _ in range(3):  # Generate 5 augmented images
            augmented_image = augment_image(image)
            augmented_features = extract_all_features(augmented_image)
            augmented_features["Label"] = label
            all_data.append(augmented_features)
    
    # Create a DataFrame from the collected features
    data = pd.DataFrame(all_data)
    
    return data

# Base path to your dataset
base_path = '/home/duyle/Documents/AIL/Aug/Q5/train'  # Update with your dataset path

# Process the directory and get the DataFrame
df = process_directory(base_path)

# Save the DataFrame to a CSV file
# df.to_csv('extracted_features.csv', index=False)

Processing Images: 100%|██████████| 2408/2408 [14:47<00:00,  2.71it/s]


In [36]:
df

,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9,...,mc_xyz_x_mode_conc,range_xyz_y_range,iqr_xyz_y_iqr,mv_xyz_y_mode,mc_xyz_y_mode_conc,range_xyz_z_range,iqr_xyz_z_iqr,mv_xyz_z_mode,mc_xyz_z_mode_conc,Label
0,589,1153,795,2285,3329,3143,1816,1405,2067,2218,...,0.103138,116,42.0,182.5,0.102234,163,93.00,179.5,0.054787,0
1,358,991,581,2498,3626,3781,1888,1413,2614,1738,...,0.109606,111,43.0,179.5,0.113865,159,99.00,177.5,0.073789,0
2,581,1174,868,2458,4088,3442,1790,1379,2000,2110,...,0.100201,129,63.0,182.5,0.103268,165,108.00,179.5,0.061740,0
3,594,1146,704,1923,2987,2588,1617,1412,2016,2213,...,0.099244,125,45.0,184.5,0.092907,165,98.00,181.5,0.057733,0
4,480,902,690,2027,3454,3024,1500,1103,1496,1739,...,0.103564,123,47.0,178.5,0.085166,167,94.00,179.5,0.045324,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,458,989,612,1711,3003,2581,1401,1122,1674,1643,...,0.129656,121,59.0,179.5,0.113927,165,111.00,177.5,0.068448,1
2404,372,881,520,1855,3893,2970,1401,1043,1662,1563,...,0.125928,138,60.0,180.5,0.118626,172,113.00,178.5,0.073700,1
2405,298,678,385,1481,2687,2403,1254,908,1555,1353,...,0.082064,80,42.0,180.5,0.082987,139,89.75,179.5,0.039686,1
2406,321,823,468,1794,3174,2976,1454,993,1628,1537,...,0.090454,100,48.0,179.5,0.097574,149,100.00,177.5,0.050897,1


In [40]:
df.to_csv('/home/duyle/Rice_photos/results/results_aug/aug_q5_train.csv', index=False)

In [ ]:
# Train SVM
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Train KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Load and process the test set for evaluation
def load_test_images(test_base_path):
    X_test = []
    y_test = []
    
    for variety in os.listdir(test_base_path):
        variety_path = os.path.join(test_base_path, variety)
        if os.path.isdir(variety_path):
            for img_name in os.listdir(variety_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(variety_path, img_name)
                    image = cv2.imread(img_path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
                    
                    features = basic_feature(image)
                    X_test.append(features)
                    y_test.append(variety)

    return np.array(X_test), np.array(y_test)

# Load and process the test set
test_base_path = '/home/duyle/Documents/AIL/Aug/test/BC-15'  # Update with your testing images path
X_test, y_test = load_test_images(test_base_path)

# Evaluate models
y_pred_svm = svm_model.predict(X_test)
y_pred_knn = knn_model.predict(X_test)

print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))
print("KNN Classification Report:\n", classification_report(y_test, y_pred_knn))